In [3]:
!pip install fastapi google-genai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached websockets-14.2-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/94.9 kB ? eta -:--:--
   ------------ --------------------------- 30.7/94.9 kB ? eta -:--:--
   ----------------- ---------------------- 41.0/94.9 kB 991.0 kB/s eta 0:00:01
   ------------------------- -------------- 61.4/94.9 kB 469.7 kB/s eta 0:00:01
   ---------------------------------------- 94.9/94.9 kB 600.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/141.0 kB ? eta -:--:--
   ----------------------- ---------------- 81.9/141.0 kB 4.8 MB/s eta 0:00:01
   ------------------------------- -------- 112.6/141.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 141.0/141.0 kB 1.2 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ---------------------------------------- 0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from google import genai
from google.genai import types
import shutil
from dotenv import load_dotenv
import os
from pydantic import BaseModel

In [2]:
load_dotenv()

True

In [3]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [24]:
import subprocess

def ffmpeg_runner(ffmpeg_code: str):
    """
    Runs an FFmpeg command and prints output in real-time.
    """
    try:
        process = subprocess.run(
            ffmpeg_code, 
            shell=True, 
            check=True, 
            stdout=subprocess.PIPE,  # Capture standard output
            stderr=subprocess.STDOUT,  # Capture errors in the same stream
            text=True  # Ensure output is treated as text (not bytes)
        )
        
        print(process.stdout)  # Print FFmpeg output
        return True
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg failed with error:\n{e.output}")
        return False

In [25]:
SYSTEM_INSTRUCTION = """
Write ffmpeg code for the queries given. The files are inside `../files/upload`. Save the file inside `../files/edit` as meow.mp4.

example code: ffmpeg -i input1.mp4 -i input2.mp4 -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" output.mp4
"""

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_INSTRUCTION,
        tools=[ffmpeg_runner],
        
    ),
)

In [62]:
response = chat.send_message("merge overlay.mp4 and the first 2 seconds of output.mp4")

ffmpeg version N-118616-g3e9777dc75-20250304 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libaribcaption --enable-libass --enable-libbluray --enable-lib

In [35]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='I apologize, but I was unable to merge the videos using the tools available to me.')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.050273236301210195, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-2.0-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=18, prompt_token_count=599, total_token_count=617), automatic_function_calling_history=[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None

In [63]:
response.automatic_function_calling_history[-1].parts[0].function_response.response['result']

True